In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [3]:
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


False

In [4]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read() # read json file
intents = json.loads(data_file)

In [5]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)# add each elements into list
        #combination between patterns and intents
        documents.append((w, intent['tag']))#add single element into end of list
        # add to tag in our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

documents

[(['Hi', 'there'], 'greeting'),
 (['How', 'are', 'you'], 'greeting'),
 (['Is', 'anyone', 'there', '?'], 'greeting'),
 (['Hey'], 'greeting'),
 (['Hola'], 'greeting'),
 (['Hello'], 'greeting'),
 (['Good', 'day'], 'greeting'),
 (['Bye'], 'goodbye'),
 (['See', 'you', 'later'], 'goodbye'),
 (['Goodbye'], 'goodbye'),
 (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'),
 (['Till', 'next', 'time'], 'goodbye'),
 (['Thanks'], 'thanks'),
 (['Thank', 'you'], 'thanks'),
 (['That', "'s", 'helpful'], 'thanks'),
 (['Awesome', ',', 'thanks'], 'thanks'),
 (['Thanks', 'for', 'helping', 'me'], 'thanks'),
 (['Help', 'me'], 'help'),
 (['I', 'need', 'assistance'], 'help'),
 (['Can', 'you', 'help', '?'], 'help'),
 (['I', "'m", 'stuck'], 'help'),
 (['Thanks'], 'thanks'),
 (['Thank', 'you'], 'thanks'),
 (['Thanks', 'a', 'lot'], 'thanks'),
 (['Tell', 'me', 'about', 'tourist', 'spots'], 'tourist_spots'),
 (['What', 'are', 'the', 'famous', 'places', 'in', 'Kolkata', '?'],
  'tourist_spots'),
 (['Tourist', 

In [ ]:
nltk.download('wordnet')

[nltk_data] Error loading wordnet: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


False

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Error loading omw-1.4: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


False

In [ ]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents\n", documents, "\n")
# classes = intents[tag]
print (len(classes), "classes\n", classes, "\n")
# words = all words, vocabulary
print (len(words), "unique lemmatized words\n", words, "\n")
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

940 documents
 [(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['Help', 'me'], 'help'), (['I', 'need', 'assistance'], 'help'), (['Can', 'you', 'help', '?'], 'help'), (['I', "'m", 'stuck'], 'help'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['Thanks', 'a', 'lot'], 'thanks'), (['Tell', 'me', 'about', 'tourist', 'spots'], 'tourist_spots'), (['What', 'are', 'the', 'famous', 'places', 'in', 'Kolkata', '?'], 'tourist_spots'), (['Tourist', 'attractions'

In [ ]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words
    pattern_words = doc[0]
    # convert pattern_words in lower case
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create bag of words array,if word match found in current pattern then put 1 otherwise 0.[row * colm(263)]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # in output array 0 value for each tag ang 1 value for matched tag.[row * colm(8)]
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle training and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test. X - patterns(words), Y - intents(tags)
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")
training

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (940, 2) + inhomogeneous part.

In [ ]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [ ]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
print("First layer:",model.layers[0].get_weights()[0])

First layer: [[-0.01370441 -0.11588766 -0.10683166 ...  0.06714636  0.0381196
   0.11222883]
 [ 0.05602863 -0.08354484  0.06900927 ... -0.03838121 -0.04807305
  -0.08269182]
 [-0.00969751 -0.02020633  0.05799077 ...  0.05497573  0.05178963
  -0.12283736]
 ...
 [ 0.06074202  0.00508358  0.03072059 ... -0.00979079  0.08900571
  -0.04602597]
 [-0.01477943 -0.10226476  0.07185936 ... -0.12294034 -0.05474468
   0.05138277]
 [-0.06508616  0.06971391 -0.01753756 ... -0.01736953 -0.12087113
   0.11301601]]


In [ ]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


Epoch 1/200
81/81 [==============================] - 1s 1ms/step - loss: 3.6252 - accuracy: 0.0296
Epoch 2/200
81/81 [==============================] - 0s 1ms/step - loss: 3.5136 - accuracy: 0.1160
Epoch 3/200
81/81 [==============================] - 0s 2ms/step - loss: 3.3421 - accuracy: 0.1556
Epoch 4/200
81/81 [==============================] - 0s 1ms/step - loss: 3.1056 - accuracy: 0.1975
Epoch 5/200
81/81 [==============================] - 0s 1ms/step - loss: 2.7976 - accuracy: 0.3012
Epoch 6/200
81/81 [==============================] - 0s 1ms/step - loss: 2.5519 - accuracy: 0.3778
Epoch 7/200
81/81 [==============================] - 0s 1ms/step - loss: 2.3028 - accuracy: 0.4074
Epoch 8/200
81/81 [==============================] - 0s 1ms/step - loss: 2.0705 - accuracy: 0.4667
Epoch 9/200
81/81 [==============================] - 0s 1ms/step - loss: 1.8324 - accuracy: 0.5185
Epoch 10/200
81/81 [==============================] - 0s 2ms/step - loss: 1.6846 - accuracy: 0.5852
Epoch 11/